In [9]:
import os
import numpy as np
import pandas as pd
import librosa
from IPython.display import clear_output

#声優の声データからMFCC（声の特徴量）を抽出し、表にまとめる。

X_data = []  # 特徴行列
y_data = []  # クラスラベルデータ
speakers_number = 7 # 声優の人数
files_number = 100 # 一人あたりのサンプルの数

for speaker_num in range(1, speakers_number + 1):  # 声優フォルダの数だけ繰り返し処理
    clear_output()
    # ボイスサンプルがあるフォルダ名
    dir_name = f'VoiceSample/{str(speaker_num).zfill(3)}'
    file_list = os.listdir(dir_name)
    for i in range(files_number):
        clear_output()
        print(f"{int((i+(speaker_num-1)*files_number)/(files_number*speakers_number)*100)}%完了")
        print(str(speaker_num) + "人目のデータを処理中・・・")
        print(f"（進行度:{i+1}/{files_number}ファイル）" )
        file_path = os.path.join(dir_name, file_list[i])  # 音声ファイルへのパス
        y, sr = librosa.load(file_path)  # 音声ファイルを読み込む
        mfcc = librosa.feature.mfcc(y, sr)  # MFCC
        mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
        mfcc = mfcc.flatten()
        mfcc = mfcc.tolist()
        mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
        X_data.append(mfcc)
        y_data.append(speaker_num)

X = pd.DataFrame(X_data, columns=[f'mfcc_{n}' for n in range(1, 13)]) # mfcc_1,mfcc_2,..のように行のタイトルをつける
y = pd.DataFrame({'target': y_data}) # targetタイトルを追加

df = pd.concat([X, y], axis=1) # 合体！！！！！！
df.to_csv('mfcc.csv', index=False)  # csvで保存
df.head() # 見せてほしいな

99%完了
7人目のデータを処理中・・・
（進行度:100/100ファイル）


/tmp/ipykernel_8879/2882801715.py:26: FutureWarning: Pass y=[-3.0901801e-06 -4.7229732e-06  1.2742006e-05 ... -3.3230467e-06
 -1.4371163e-06  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y, sr)  # MFCC


,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,target
0,79.305939,-2.388980,28.821922,8.106466,-7.335474,-5.098837,2.708056,1.108589,-6.039336,-11.022418,8.562423,-8.208080,1
1,46.389778,15.316972,29.470976,15.136189,7.453907,-4.679848,6.583255,-3.255087,2.030280,-12.806031,9.289038,-7.932013,1
2,97.034988,19.946007,42.861015,15.739766,-5.527338,0.597292,3.899351,-1.708565,-6.449223,-6.763590,11.951451,-12.204566,1
3,84.402969,14.758104,36.634586,19.574112,-5.123018,-3.361386,4.112405,-0.293475,-1.699620,-10.678855,12.195671,-13.852660,1
4,94.733459,15.617436,38.703354,16.677921,-3.071551,-1.830640,5.379351,0.139020,-8.512769,-13.017569,8.190866,-15.577478,1


In [14]:

import pandas as pd
from pycaret.regression import *

# 機械学習前の下準備を行う。以下、Pycaretの関数を使う場合このセルを実行してから使うこと
print("機械学習の準備中・・・")

mfcc_data = pd.read_csv("mfcc.csv") # MFCCのデータを読み込む
# データの前準備(以下、Pycaretの関数を使う場合setupを呼び出してから使うこと)
reg = setup(data=mfcc_data, target='target', data_split_shuffle=True, use_gpu=True, silent=True, fold=5, n_jobs=-1)

print("準備完了")

# 機械学習のモデルを複数比較し、精度が高かったモデルを表示する。
print("最適モデルを検索中・・・")

compare_models()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.8543,1.4171,1.1903,0.6434,0.2759,0.3343,2.3240
rf,Random Forest Regressor,0.8802,1.5283,1.2352,0.6158,0.2833,0.3409,2.7740
gbr,Gradient Boosting Regressor,0.9651,1.7196,1.3108,0.5686,0.3027,0.3734,0.4080
lightgbm,Light Gradient Boosting Machine,0.9571,1.7526,1.3227,0.5597,0.3045,0.3645,0.4860
knn,K Neighbors Regressor,0.9351,1.7810,1.3335,0.5512,0.3031,0.3624,0.2280
ada,AdaBoost Regressor,1.0784,1.8743,1.3685,0.5295,0.3239,0.4520,0.3720
br,Bayesian Ridge,1.1108,1.9967,1.4123,0.4984,0.3252,0.4332,0.0520
ridge,Ridge Regression,1.1122,2.0034,1.4143,0.4973,0.3256,0.4314,0.0600
lar,Least Angle Regression,1.1122,2.0034,1.4143,0.4973,0.3256,0.4314,0.0700
lr,Linear Regression,1.1122,2.0034,1.4143,0.4973,0.3256,0.4314,0.1100


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=3307, verbose=0, warm_start=False)

In [20]:
from pycaret.regression import *
# 最も精度が高かったモデルを生成
tuned_model = 0

print("モデルを作成中・・・")
model = create_model('et', verbose=False) # モデルを生成
#print("モデルを調整中・・・")
#tuned_model = tune_model(model, optimize="R2") # モデルを調整...すると精度落ちるの何...しないほうがいいのか？
print("テストデータで予測中・・・")
predict_data = predict_model(tuned_model or model) # 既存のデータで予測
predict_data.to_csv('predict_result.csv', index=False)  # csvで保存

final_model = finalize_model(model)
save_model(final_model,'Final Model')#モデルを保存


モデルを作成中・・・
テストデータで予測中・・・


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.8275,1.3275,1.1522,0.6677,0.2744,0.3449


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='target',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy...
                  ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                    

In [6]:
from pycaret.regression import *

final_model = load_model('Final Model') # モデルの読み込み

Transformation Pipeline and Model Successfully Loaded


In [3]:
import sounddevice as sd
import wave
import numpy as np

#音声を録音

FILE_NAME = 'my_recording.wav'  # 保存するファイル名
wave_length = 5  # 録音する長さ（秒）
sample_rate = 16_000  # サンプリング周波数

print("録音開始")
# 録音開始（wave_length秒間録音。wait で録音し終わるまで待つ）
data = sd.rec(int(wave_length * sample_rate), sample_rate, channels=1)
sd.wait()
print("録音終了")

# ノーマライズ。量子化ビット16bitで録音するので int16 の範囲で最大化する
data = data / data.max() * np.iinfo(np.int16).max

# float -> int
data = data.astype(np.int16)

# ファイル保存
with wave.open(FILE_NAME, mode='wb') as wb:
    wb.setnchannels(1)  # モノラル
    wb.setsampwidth(2)  # 16bit=2byte
    wb.setframerate(sample_rate)
    wb.writeframes(data.tobytes())  # バイト列に変換

録音開始
録音終了


/tmp/ipykernel_8879/3780763905.py:18: RuntimeWarning: invalid value encountered in true_divide
  data = data / data.max() * np.iinfo(np.int16).max


MFCCを抽出しています・・・
処理完了


/tmp/ipykernel_8879/3780763905.py:43: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y, sr)  # MFCC


In [25]:
import pandas as pd
import librosa
import numpy as np

###MFCCを抽出###
print("MFCCを抽出しています・・・")

data = []
TEST_FILE_NAME = "TestSample/hanae2.wav"

test = True

y, sr = librosa.load(TEST_FILE_NAME if test else FILE_NAME)  # 音声ファイルを読み込む
mfcc = librosa.feature.mfcc(y, sr)  # MFCC
mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
mfcc = mfcc.flatten()
mfcc = mfcc.tolist()
mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
data.append(mfcc)

df = pd.DataFrame(data, columns=[f'mfcc_{n}' for n in range(1, 13)])

df.to_csv('mfcc_my_recording.csv', index=False)  # csvで保存
df.head()


MFCCを抽出しています・・・


,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12
0,69.788544,-11.775072,20.867018,-1.403565,2.142157,-3.157141,4.668924,-11.96955,8.3155,-12.742859,4.376723,-9.088756


In [26]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
from pycaret.regression import *

#録音した音声データからどの声優に似ているかを特定

actors = ["下野紘", "花江夏樹", "梶裕貴", "沢城みゆき", "鬼頭明里", "水瀬いのり", "悠木碧"]

predict_my_recording = predict_model(final_model, data = pd.read_csv("mfcc_my_recording.csv")) # 未知データを予測
predict_label = predict_my_recording.at[0, "Label"] # 予測データを取得（小数点数）
voice_actor_number = Decimal(str(predict_label)).quantize(Decimal('0'), rounding=ROUND_HALF_UP) # 整数に四捨五入
print("あなたの声は" + actors[int(voice_actor_number-1)] + "に似ています。")
print("予測値：" + str(predict_label))

あなたの声は梶裕貴に似ています。
予測値：2.72


In [2]:
import os
import numpy as np
import pandas as pd
import librosa

#声優の声データからMFCC（声の特徴量）を抽出し、表にまとめる。

X_data = []  # 特徴行列
speakers_number = 7 # 声優の人数
files_number = 5 #声優一人あたりのファイル数

for speaker_num in range(1, speakers_number + 1):  # 声優フォルダの数だけ繰り返し処理
    # ボイスサンプルがあるフォルダ名
    dir_name = f'TestSample/{str(speaker_num).zfill(3)}'
    file_list = os.listdir(dir_name)
    for i in range(files_number):
        print(f"{int((i+(speaker_num-1)*files_number)/(files_number*speakers_number)*100)}%完了")
        print(str(speaker_num) + "人目のデータを処理中・・・")
        print(f"（進行度:{i+1}/{files_number}ファイル）" )
        file_path = os.path.join(dir_name, file_list[i])  # 音声ファイルへのパス
        y, sr = librosa.load(file_path)  # 音声ファイルを読み込む
        mfcc = librosa.feature.mfcc(y, sr)  # MFCC
        mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
        mfcc = mfcc.flatten()
        mfcc = mfcc.tolist()
        mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
        X_data.append(mfcc)

df = pd.DataFrame(X_data, columns=[f'mfcc_{n}' for n in range(1, 13)]) # mfcc_1,mfcc_2,..のように行のタイトルをつける

df.to_csv('mfcc_test.csv', index=False)  # csvで保存
df.head() # 見せてほしいな

0%完了
1人目のデータを処理中・・・
（進行度:1/5ファイル）


EOFError: 

In [1]:
from pycaret.regression import *
import numpy as np
import pandas as pd

actors = ["下野紘", "花江夏樹", "梶裕貴", "沢城みゆき", "鬼頭明里", "水瀬いのり", "悠木碧"]
speakers_number = 7

final_model = load_model('Final Model')
results = []

#学習に使用していない声優の声データを用いて正答率を出す。
testFolder = "TestSample"
folders = os.listdir(testFolder)

for speaker_num in range(1, speakers_number):
    files = os.listdir(f'{testFolder}/{str(speaker_num).zfill(3)}')
    for testFile in files:
        print(f'{speaker_num}/{speakers_number}人完了')
        predict_data = predict_model(final_model, data = pd.read_csv("mfcc_my_recording.csv")) # 未知データを予測
        predict_label = predict_my_recording.at[0, "Label"] # 予測データを取得（小数点数）
        voice_actor_number = Decimal(str(predict_label)).quantize(Decimal('0'), rounding=ROUND_HALF_UP) # 整数に四捨五入
        result = [actors[voice_actor_number], predict_label, f'{actors[speaker_num]}({speaker_num})']
        print("result")
        results.append(result)

df = pd.DataFrame(data, columns=["予想した声優", "予測値", "正解値"])
df.to_csv("test_result.csv", index=False)
df.head()

AttributeError: 'list' object has no attribute 'length'